In [127]:
from __future__ import absolute_import, division, print_function, unicode_literals

import collections
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing import sequence

import tensorflow as tf

from tensorflow.keras import layers

from keras.utils.vis_utils import plot_model

from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model
from keras.datasets import imdb

# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=20000)
np.load = np_load_old
# Do we need nested inputs? https://www.tensorflow.org/guide/keras/rnn#rnns_with_listdict_inputs_or_nested_inputs
# Provides function for sliding window for training data of time series data https://www.tensorflow.org/tutorials/structured_data/time_series#the_weather_dataset
# How do we use our LSTM model as an input to a dense model with member features?

# Reference for classification https://keras.io/examples/imdb_lstm/
# Look into sequence.pad_sequences for padding our data
# IMDB data is padded and is an array of lists

In [135]:
batch_size = 10
num_batch = 10
max_timestep = 100
input_dim = 8
epochs = 5

# Random input data of variable timestep length
x_train = []
for i in range(num_batch*batch_size):
    timestep = np.random.randint(200)
    x_train.append(np.random(size=(timestep,input_dim))
x_train = np.array(x_train)

y_train = np.random.binomial(n=1,p=.5,size=(batch_size * num_batch,1))

model = tf.keras.Sequential()
model.add(layers.LSTM(units=12, input_length=timestep, input_dim=input_dim))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(1, activation='softmax'))

model.summary()
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,verbose=1)


TypeError: 'float' object cannot be interpreted as an integer

In [64]:
x_pred = np.random.random((10, timestep, input_dim))
model.predict(x_pred) # Why does it always result in a prediction of 1?

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)